In [0]:
%flink.ssql
DROP TABLE IF EXISTS input_table;
CREATE TABLE input_table (
  InvoiceNo STRING,
  Customer STRING,
  InvoiceDate STRING,
  Quantity STRING,
  Description STRING,
  UnitPrice STRING,
  Country STRING,
  StockCode STRING
) WITH (
  'connector' = 'kinesis',
  'stream' = 'acmecoOrders',
  'aws.region' = 'us-east-1',
  'aws.endpoint' = 'https://vpce-01a4f644e54dd6b87-upj1u2gs.kinesis.us-east-1.vpce.amazonaws.com',

  'aws.clientconfig.connectionTimeout' = '5000',
  'aws.clientconfig.requestTimeout'    = '20000',
  'aws.clientconfig.maxConnections'    = '200',

  'scan.shard.getrecords.intervalmillis' = '100',
  'scan.stream.initpos' = 'LATEST',
  'aws.credentials.provider' = 'AUTO',

  'format' = 'json',
  'json.ignore-parse-errors' = 'true',
  'json.fail-on-missing-field' = 'false'
);

In [1]:
%flink.ssql
DROP TABLE IF EXISTS output_table;
CREATE TABLE output_table (
  StockCode STRING,
  InvoiceDate STRING,
  Quantity INT,
  Description STRING,
  UnitPrice DOUBLE,
  Country STRING
) WITH (
  'connector' = 'kinesis',
  'stream' = 'alertsStream',
  'aws.region' = 'us-east-1',
  'aws.endpoint' = 'https://vpce-01a4f644e54dd6b87-upj1u2gs.kinesis.us-east-1.vpce.amazonaws.com',

  'aws.clientconfig.connectionTimeout' = '5000',
  'aws.clientconfig.requestTimeout'    = '20000',
  'aws.clientconfig.maxConnections'    = '200',
  'aws.credentials.provider' = 'AUTO',

  'format' = 'json',
  'json.ignore-parse-errors' = 'true',
  'json.fail-on-missing-field' = 'false'
);


In [2]:
%flink.ssql

SELECT
  InvoiceNo,
  CAST(Customer AS BIGINT) AS Customer,
  InvoiceDate,
  CAST(Quantity AS INT) AS Quantity,
  Description,
  CAST(UnitPrice AS DOUBLE) AS UnitPrice,
  Country,
  StockCode
FROM input_table;


In [ ]:
%flink.ssql
INSERT INTO output_table
SELECT
  StockCode,
  InvoiceDate,
  CAST(Quantity AS INT) AS Quantity,
  Description,
  CAST(UnitPrice AS DOUBLE) AS UnitPrice,
  Country
FROM input_table WHERE CAST(Quantity AS INT) < 1;


In [4]:
%flink.ssql
